In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import json
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
tf.compat.v1.Session()
import keras
import keras_metrics

import import_ipynb
from reading_splitting_dataset_functions import *
from __future__ import print_function
from network import BasicDenseFeedforwardNet
from solver import Solver
from functools import partial
from tensorflow.keras.optimizers import SGD, RMSprop, Adam

importing Jupyter notebook from reading_splitting_dataset_functions.ipynb
importing Jupyter notebook from network.ipynb
importing Jupyter notebook from solver.ipynb
importing Jupyter notebook from optim.ipynb


In [2]:
data_roi=open_js_file('data_preprocessed_roi.JSON')
print('length ROI:', len(data_roi))

length ROI: 33676


In [3]:
df_roi, fid_roi, v_roi, lva_roi, lha_roi = get_acceleration_fid_v_labels(data_roi)
l_roi = labels_roi(lva_roi, lha_roi)
print(l_roi)

[2. 2. 2. ... 3. 3. 3.]


In [5]:
#def run experiment(repeats = 5):
repeats = 5
test_loss = np.zeros(repeats)
test_acc = np.zeros(repeats)
training_loss = np.zeros(repeats)
training_acc = np.zeros(repeats)
before = []
for j in range(repeats):
    print('repeat:', j)
    x_train, y_train, x_test, y_test, before=train_test_split_cross_validation(fid_roi, df_roi, l_roi, before)
    x_train, y_train, x_test, y_test = bring_in_right_shape_self(x_train, y_train, x_test, y_test)
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(64, activation='tanh'),
        tf.keras.layers.GaussianDropout(0.5),
        #tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')
    ])

    opt = Adam(.0010)
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    model.compile(loss=loss_fn, optimizer=opt, metrics=["accuracy", keras_metrics.precision(), keras_metrics.recall()])
    number_epochs=30
    H = model.fit(x_train, y_train, batch_size=128, validation_data=(x_test, y_test), epochs=number_epochs)
    
    test_loss[j] = H.history["val_loss"][number_epochs-1]
    test_acc[j] = H.history["val_accuracy"][number_epochs-1]

    training_loss[j] = H.history["loss"][number_epochs-1]
    training_acc[j] = H.history["accuracy"][number_epochs-1]
    

print('Test loss: ', test_loss)
print('Training loss: ', training_loss)
print('Test accuracy: ', test_acc)
print('Training accuracy: ', training_acc)
print('Mean of test accuracy: ', np.mean(test_acc), 'Variance of test accuracy: ', np.var(test_acc))
print('Mean of training accuracy: ', np.mean(training_acc), 'Variance of training accuracy: ', np.var(training_acc))

repeat: 0
Train on 26374 samples, validate on 7302 samples
Epoch 1/30
26374/26374 [==============================] - 3s 96us/sample - loss: 1.0243 - accuracy: 0.7292 - precision: 0.7777 - recall: 0.3888 - val_loss: 0.9937 - val_accuracy: 0.7609 - val_precision: 0.8522 - val_recall: 0.5386
Epoch 2/30
26374/26374 [==============================] - 2s 94us/sample - loss: 0.9145 - accuracy: 0.8423 - precision: 0.9146 - recall: 0.6621 - val_loss: 0.9538 - val_accuracy: 0.7931 - val_precision: 0.8303 - val_recall: 0.6469
Epoch 3/30
26374/26374 [==============================] - 2s 90us/sample - loss: 0.8816 - accuracy: 0.8719 - precision: 0.9402 - recall: 0.7243 - val_loss: 0.9375 - val_accuracy: 0.8088 - val_precision: 0.8606 - val_recall: 0.6544
Epoch 4/30
26374/26374 [==============================] - 2s 87us/sample - loss: 0.8644 - accuracy: 0.8870 - precision: 0.9346 - recall: 0.7566 - val_loss: 0.9312 - val_accuracy: 0.8111 - val_precision: 0.8342 - val_recall: 0.7168
Epoch 5/30
26374/